- eks의 구성조건, pod network, aws controller 등의 설치 과정이 node 숫자의 변화에 민감하고,   
- infra 구성 완료 전에 일어 나면 안되고  
- infra 구성 후 node 숫자를 조절 하면서 실행할 필요가 있어 이를 ad-hoc script로 남깁니다.

# 클러스터 구성 과정
# 1. node 0개로 클러스터 생성 (secondary cidr 설정을 위해)
# 2. bootstrap (secondary cidr, aws load balancer controller)
# 3. 노드 갯수 원하는 양으로 조절
## 의 과정을 거쳐야 한다. (aws는 k8s에 최적화가 덜되서...)

Deprecated (이전 절차)
절차

1. node_size.yml => 0
2. terraform apply => output으로 eniconfig.yaml 파일 작성
3. aws eks --region us-east-1 update-kubeconfig --name test-eks-cluster
4. kubectl create -f ./k8s-manifests/eni-configs/.
5. kubectl set env daemonset aws-node -n kube-system AWS_VPC_K8S_CNI_CUSTOM_NETWORK_CFG=true
6. kubectl set env daemonset aws-node -n kube-system ENI_CONFIG_LABEL_DEF=failure-domain.beta.kubernetes.io/zone
7. node_size.yml => 0
8. terraform apply
9. node_size.yml => cluster size you want
10. terraform apply

절차

1. node_size.yml => 0
2. terraform apply => output으로 eniconfig.yaml 파일 작성
3. boot strap 과정에 개편을 거쳐서 이젠 boot strap 데이터를 terraform 으로 전송해서, null_resource를 통해 실행시킨다.
4. node_size.yml => cluster size you want
5. terraform apply --auto-approve
6. node_size.yml => 원하는 사이즈
7. terraform apply --auto-approve

여기다가 초기 정보 (cluster name, region) 등 몰아서 설정하게 해야 되는데...

뭐... node0 일때는 kubeflow 실행용 null resource 파일을 a.tf.txt로 했다가
node1로 늘린 다음에 a.tf로 바꿔서 실행시켜야 되나...

In [1]:

"""
kubeflow 를 설치후 node=0 로 만들고 다시 키니까
=> block storage가 떨어진 후 다시 안붙음
=> 어? 왜 주소가 달라 너 안붙여줄거야
=> 아니 나라니까?
=> 그럼 다시 만들기라도 해야 되는데 안만들고 그냥 에러남
=> secondary cidr와 install kubeflow의 bootstrap을 쪼개기로 결정
"""

import os
def change_current_state_for_bootstrap(state="secondary_cidr"):
    # There is two state, making secondary cidr for pod
    # Another is bootstrap for aws controllers and kubeflow

    if state == "secondary_cidr" :
        if os.path.isfile("bootstrap_ctrl_and_kf.tf"):
            os.rename("bootstrap_ctrl_and_kf.tf", "bootstrap_ctrl_and_kf.tf.txt")
        else :
            print("file already exist")
    elif state == "bootstrap" :
        if os.path.isfile("bootstrap_ctrl_and_kf.tf.txt"):
            os.rename("bootstrap_ctrl_and_kf.tf.txt", "bootstrap_ctrl_and_kf.tf")
        else :
            print("file already exist")
    else:
        print("something wrong in bootstrap file name")

In [2]:
#!terraform destroy --auto-approve

# 1. 초기 클러스터 크기

In [3]:
import yaml
def change_node_size_yml(desired_size, max_size, min_size):
    with open('node_size.yml') as f:
        node_size = yaml.load(f, Loader=yaml.FullLoader)
        print('current_state',node_size)
        
    node_size['desired_size'] = desired_size
    node_size['min_size'] = min_size
    node_size['max_size'] = max_size
    
    print('after change', node_size)
    
    with open('node_size.yml', 'w') as f:
        yaml.dump(node_size,f)

In [4]:
desired_size = 0
max_size = 1 #maxsize는 0이 될 수 없다.
min_size = 0

In [5]:
change_node_size_yml(desired_size, max_size, min_size)

current_state {'desired_size': 0, 'max_size': 1, 'min_size': 0}
after change {'desired_size': 0, 'max_size': 1, 'min_size': 0}


In [6]:
change_current_state_for_bootstrap(state="secondary_cidr") 

# 2. Terraform 실행

#테라폼 설치 법 2023-06-25 현재 최신버전  1.5.1    
https://askubuntu.com/questions/983351/how-to-install-terraform-in-ubuntu

In [7]:
#mv terraform /usr/local/bin #컨테이너 안에서 실행했음

In [8]:
#!terraform --version

In [9]:
#!terraform init

In [10]:
#!terraform plan

In [11]:
!terraform apply --auto-approve

data.http.myip: Reading...
data.http.myip: Read complete after 0s [id=http://ipv4.icanhazip.com]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create
 <= read (data resources)

Terraform will perform the following actions:

  # data.template_file.eniconfig will be read during apply
  # (config refers to values not yet known)
 <= data "template_file" "eniconfig" {
      + id       = (known after apply)
      + rendered = (known after apply)
      + template = <<-EOT
            #https://cj-hewett.medium.com/using-templates-in-terraform-to-generate-kubernetes-yaml-5f60cfa0109
            #https://registry.terraform.io/providers/hashicorp/template/latest/docs/data-sources/file
            #https://registry.terraform.io/providers/hashicorp/local/latest/docs/resources/file
            #https://stackoverflow.com/questions/63845957/terraform-saving-output-to-file
            #https://github.com/h

In [12]:
import time

time.sleep(120)

# 3. bastion server에서 bootstrap 실행
# => 테라폼 null_resource로 대체

# 4 node size -> 0 으로 #곧 deprecated됩니다.

In [13]:
#한번에 잘 안없어지더라?...
#desired_size = 0
#max_size = 1 #maxsize는 0이 될 수 없다.
#min_size = 0

In [14]:
#change_node_size_yml(desired_size, max_size, min_size)

In [15]:
#!terraform apply --auto-approve

# 5 node size -> 다시 늘리기

In [16]:
#node_size.yml => cluster size you want

In [28]:
#노드 그룹당 1개 , 노드그룹은 terraform에 2개로 설정되어 있음
#리소스 때문에 kubeflow 설치 못하는거 같아서 2배로 늘려봄
desired_size = 2
max_size = 2
min_size = 2

change_node_size_yml(desired_size, max_size, min_size)

current_state {'desired_size': 2, 'max_size': 2, 'min_size': 2}
after change {'desired_size': 2, 'max_size': 2, 'min_size': 2}


In [29]:
change_current_state_for_bootstrap(state="bootstrap") 

file already exist


In [31]:
!terraform apply --auto-approve

data.http.myip: Reading...
data.http.myip: Read complete after 0s [id=http://ipv4.icanhazip.com]
aws_iam_instance_profile.bastion-ec2-instance-profile: Refreshing state... [id=bastion-ec2-instance-profile]
aws_vpc.kube_vpc: Refreshing state... [id=vpc-006367fff3cb10f58]
aws_iam_role.iam-role-eks-cluster: Refreshing state... [id=iam-role-eks-cluster]
aws_iam_role.iam-role-eks-nodegroup: Refreshing state... [id=iam-role-eks-nodegroup]
aws_efs_file_system.efs-file-system: Refreshing state... [id=fs-010dc5dabb3dd1cec]
aws_iam_policy.alb-iam-policy: Refreshing state... [id=arn:aws:iam::222170749288:policy/alb-iam-policy]
aws_iam_role.iam-role-ec2-instance-bastion: Refreshing state... [id=iam-role-ec2-instance-bastion]
aws_iam_role_policy_attachment.iam-policy-eks-nodegroup-cni: Refreshing state... [id=iam-role-eks-nodegroup-20230704114822933100000003]
aws_iam_role_policy_attachment.iam-policy-eks-nodegroup-ecr: Refreshing state... [id=iam-role-eks-nodegroup-20230704114822976900000006]
aws_i

In [20]:
#잠깐 쉴때는 노드 사이즈만 줄여도 괜찮네
#!terraform apply --auto-approve

In [21]:
여기 이전에 멈춰~~~~

SyntaxError: invalid syntax (2286233506.py, line 1)

In [ ]:
!terraform destroy --auto-approve

In [ ]:
야... 이거 주피터 하나 돌렸더니 사양 딸려서 멈춘거 같은데?

모니터링이 안되니까 답답해가지고....